In [1]:
import os
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields
import numpy as np
import pandas as pd
from pprint import pprint
from dateutil.relativedelta import relativedelta
import json

In [2]:
dirname = "data/2024/2"
dfNum = pd.read_table(f"{dirname}/num.txt",delimiter="\t", low_memory=False)
dfPre = pd.read_table(f"{dirname}/pre.txt",delimiter="\t", low_memory=False)
dfSub = pd.read_table(f"{dirname}/sub.txt",delimiter="\t", low_memory=False)
dfTag = pd.read_table(f"{dirname}/tag.txt",delimiter="\t", low_memory=False)
dfSym = pd.read_table(f"{dirname}/ticker.txt", delimiter="\t", header=None, names=['symbol','cik'], low_memory=False)        


In [3]:
class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [5]:
def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    try:
        periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    except ValueError:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Period: "+str(submitted["period"]))
        continue       
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        #FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IS':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 291 Dtos size: 221 Bs: 87 Cf: 46 Ic: 88 Json size: 61kb.
file 1 of 7676 exportFiles/data/2024/2/0000002178-24-000054.json stored in 5353.026ms.
FilterDfNum size: 247 Dtos size: 183 Bs: 70 Cf: 55 Ic: 58 Json size: 51kb.
file 2 of 7676 exportFiles/data/2024/2/0000002488-24-000056.json stored in 4477.103ms.
FilterDfNum size: 786 Dtos size: 451 Bs: 136 Cf: 87 Ic: 228 Json size: 125kb.
file 3 of 7676 exportFiles/data/2024/2/0000002969-24-000026.json stored in 13763.548ms.
FilterDfNum size: 204 Dtos size: 139 Bs: 73 Cf: 42 Ic: 24 Json size: 38kb.
file 4 of 7676 exportFiles/data/2024/2/0000003499-24-000015.json stored in 4062.469ms.
FilterDfNum size: 427 Dtos size: 315 Bs: 100 Cf: 76 Ic: 139 Json size: 91kb.
file 5 of 7676 exportFiles/data/2024/2/0000003545-24-000066.json stored in 7667.548ms.
FilterDfNum size: 426 Dtos size: 260 Bs: 148 Cf: 62 Ic: 50 Json size: 92kb.
file 6 of 7676 exportFiles/data/2024/2/0000003570-24-000055.json stored in 7575.063ms.
FilterDfNum size: 378

KeyboardInterrupt: 